# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [4]:
# Verify that loading works, for one example
n_samples = 5

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))
# clean_input, corrupted_input, labels = counterfact_dataset.get_single_sample(0)

# print(clean_input)
# print(corrupted_input)
# print(labels)


collate tensor([[[24111],
         [15823]]], device='cuda:0')


In [26]:
from applications.pipeline import localise_models

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=n_samples)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))

target_mlp, target_attn = localise_models(model, clean_input, corrupted_input, labels, overwrite=False)

collate tensor([[[24111],
         [15823]]], device='cuda:0')
torch.Size([10, 12, 3072])


In [28]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

evaluation_scores = defaultdict(list)
counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

for n, (clean_input, corrupted_input, labels) in enumerate(counterfact_dataloader):

    original_output = model.generate(clean_input, max_new_tokens=5, do_sample=False)

    edited_model = edit_model(model, clean_input, corrupted_input, labels, target_mlp[n], target_attn[n], n_epochs=10)

    # Evaluate
    print(f"Prompt: {clean_input}")
    print("Original output:", original_output)
    # print(f"Original answer: {labels[:, 0]}. Target answer: {labels[:, 1]}")

    print(edited_model.generate(clean_input, max_new_tokens=5, do_sample=False))

    score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
    evaluation_scores["Efficacy score"].append(score.item())
    evaluation_scores["Efficacy magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
    evaluation_scores["Generalisation score"].append(score.item())
    evaluation_scores["Generalisation magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
    evaluation_scores["Specificity score"].append(score.item())
    evaluation_scores["Specificity magnitude"].append(magnitude.item())

    consistency_score = evaluate_consistency(model, n, verbose=False)
    evaluation_scores["Consistency score"].append(score.item())
    evaluation_scores["Consistency magnitude"].append(magnitude.item())

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

collate tensor([[[24111],
         [15823]]], device='cuda:0')


100%|██████████| 5/5 [00:00<00:00, 54.36it/s]


Fine tuning model...
Target MLP tensor(656, device='cuda:0')
Target attn tensor(17, device='cuda:0')


Total loss: 10.913865089416504, forget loss: 11.370841979980469, retain loss: 10.761539459228516
Total loss: 8.81628131866455, forget loss: 12.037135124206543, retain loss: 7.7426629066467285
Total loss: 7.5256171226501465, forget loss: 10.85511589050293, retain loss: 6.415783882141113
Total loss: 4.503967761993408, forget loss: 8.474865913391113, retain loss: 3.18033504486084
Total loss: 1.5352414846420288, forget loss: 6.140965938568115, retain loss: 0.0
Total loss: 0.9295044541358948, forget loss: 3.718017816543579, retain loss: 0.0
Total loss: 0.8976931571960449, forget loss: 1.5604335069656372, retain loss: 0.6767797470092773
Total loss: 0.35864177346229553, forget loss: 1.4345670938491821, retain loss: 0.0
Total loss: 1.2953824996948242, forget loss: 0.5293347835540771, retain loss: 1.5507316589355469
Total loss: 0.9146296381950378, forget loss: 0.7651317119598389, retain loss: 0.9644622802734375
Prompt: ['The mother tongue of Danielle Darrieux is']
Original output: The mother to

100%|██████████| 5/5 [00:00<00:00, 55.14it/s]


...

100%|██████████| 5/5 [00:00<00:00, 54.71it/s]



Fine tuning model...
Target MLP tensor(946, device='cuda:0')
Target attn tensor(6, device='cuda:0')
Total loss: 16.204971313476562, forget loss: 16.436264038085938, retain loss: 16.12787437438965
Total loss: 12.382894515991211, forget loss: 13.323369979858398, retain loss: 12.069402694702148
Total loss: 9.707677841186523, forget loss: 10.942668914794922, retain loss: 9.296014785766602
Total loss: 7.083451271057129, forget loss: 8.874706268310547, retain loss: 6.486366271972656
Total loss: 4.930303573608398, forget loss: 7.290166854858398, retain loss: 4.143682479858398
Total loss: 4.85600471496582, forget loss: 6.13484001159668, retain loss: 4.429726600646973
Total loss: 3.7668380737304688, forget loss: 5.87274694442749, retain loss: 3.064868688583374
Total loss: 2.6945037841796875, forget loss: 5.769637584686279, retain loss: 1.6694592237472534
Total loss: 2.744269371032715, forget loss: 5.620643138885498, retain loss: 1.7854783535003662
Total loss: 2.866326332092285, forget loss: 5.

 40%|████      | 2/5 [00:00<00:00, 36.14it/s]


What is the twin city of Lyon? It isManila
Original label: Beirut
Target label: Manila
Outputs: [' its', ' its', ' freedom', ' freedom', ' freedom', ' freedom', ' its', ' the', ' the', ' the']


100%|██████████| 5/5 [00:00<00:00, 46.56it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [20]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,1.0,0.000022,1.0,0.000059,0.8,0.000020,0.8,0.000020
1,0.0,-0.000009,0.0,-0.000006,0.8,0.001472,0.8,0.001472
2,1.0,0.000090,1.0,0.000065,0.9,0.000102,0.9,0.000102
3,1.0,0.000635,1.0,0.011301,0.9,0.000405,0.9,0.000405
4,0.3,0.000045,0.5,0.000120,1.0,0.003104,1.0,0.003104
